In [10]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import constants as cc, units as uu

# Constants
G = cc.G.si.value  # gravitational constant in m^3 kg^-1 s^-2
M = cc.M_sun.si.value  # mass of the Sun in kg
c = cc.c.si.value  # speed of light in m/s
GM = G*M  # gravitational parameter for the Sun in m^3/s^2

# Constants for Mercury
a_mercury = 57.909e9 * uu.m  # Semi-major axis of Mercury's orbit in meters
e = e_mercury = 0.205630  # Eccentricity of Mercury's orbit

# Specific angular momentum (h) for Mercury
# h = sqrt(G * M * a * (1 - e^2))
h_mercury = ((cc.G * cc.M_sun * a_mercury * (1 - e_mercury**2))**0.5).decompose()
h = h_mercury.to(uu.m**2 / uu.s).value  # Convert to m^2/s for consistency with previous units used




h0 2712984358777611.0


In [12]:
def r_derivatives(theta, h, e, GM, x):
    sigma = x[0]
    k = x[1]
    # Expression for r, r_dot, and r_double_dot
    r = (h ** 2 / GM) / (1 + k * e * np.cos(theta / sigma))
    r_dot = -GM*e*np.sin(theta/sigma)/(h*sigma)
    r_double_dot =  GM**3*e*(e*np.cos(theta/sigma) + 1)**2*np.cos(theta/sigma)/(h**4*sigma**2)
    return r, r_dot, r_double_dot


def error_function(x):
    def integrand(theta):
        r, r_dot, r_double_dot = r_derivatives(theta, h, e, GM, x)
        theta_dot = h / r**2
        v_squared = r_dot**2 + r**2 * theta_dot**2
        c = 299792458
        c_squared = c**2
        vc = v_squared/c_squared
        if(vc>1):
            print(vc, "FTL")
        a_theoretical = -GM*(1-v_squared/c**2)**0.5 / r**2 / (1 + 0.5*(r_dot / c)**2)**2
        a_numerical = r_double_dot - r * theta_dot ** 2  # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2
    sigma = x[0]
    integral_error, _ = quad(integrand, 0, 2*np.pi/sigma, epsabs=1e-14, epsrel=1e-14)
    return 100*integral_error

# Initial guess for sigma
initial_x = [0.9, 1.0, h]
result = minimize(error_function, initial_x, method='Nelder-Mead', tol=1E-14)
print("Optimized sigma:", result.x)


# Constants
orbital_period_days = 88  # Orbital period of Mercury in days
days_per_year = 365.25  # Average number of days per year, accounting for leap years
years_per_century = 100  # Number of years in a century
optimized_sigma = result.x[0] # The optimized value for sigma
optimized_eccentricity = result.x[1]*e

# Calculate the number of orbits Mercury completes in a century
orbits_per_century = (days_per_year / orbital_period_days) * years_per_century

# Calculate the precession per orbit in radians
precession_per_orbit_radians = 2 * np.pi * (optimized_sigma - 1)

# Convert precession per orbit to arcseconds
precession_per_orbit_arcseconds = np.degrees(precession_per_orbit_radians) * 3600

# Total precession in arcseconds per century
total_precession_arcseconds_per_century = precession_per_orbit_arcseconds * orbits_per_century
total_precession_arcseconds_per_century 


Optimized sigma: [1.00000009e+00 9.99999785e-01 2.42073980e+15]


46.840542243327526

In [13]:

def r_derivatives(theta, e, GM, x):
    sigma = x[0]
    k = x[1]
    h = x[2]
    # Expression for r, r_dot, and r_double_dot
    r = (h ** 2 / GM) / (1 + k * e * np.cos(theta / sigma))
    r_dot = -GM*e*np.sin(theta/sigma)/(h*sigma)
    r_double_dot =  GM**3*e*(e*np.cos(theta/sigma) + 1)**2*np.cos(theta/sigma)/(h**4*sigma**2)
    return r, r_dot, r_double_dot


def error_function(x):
    def integrand(theta):
        r, r_dot, r_double_dot = r_derivatives(theta, e, GM, x)
        h= x[2]
        theta_dot = h / r**2
        v_squared = r_dot**2 + r**2 * theta_dot**2
        c = 299792458
        c_squared = c**2
        vc = v_squared/c_squared
        if(vc>1):
            print(vc, "FTL")
        a_theoretical = -GM*(1-v_squared/c**2)**0.5 / r**2 / (1 + 0.5*(r_dot / c)**2)**2
        a_numerical = r_double_dot - r * theta_dot ** 2  # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2

    integral_error, _ = quad(integrand, 0, 2*np.pi/sigma, epsabs=1e-14, epsrel=1e-14)
    return 100*integral_error

# Initial guess for sigma
initial_x = [0.9, 1.0,h]
result = minimize(error_function, initial_x, method='Nelder-Mead', tol=1E-14)
print("Optimized sigma:", result.x)


# Constants
orbital_period_days = 87.969  # Orbital period of Mercury in days
days_per_year = 365.25  # Average number of days per year, accounting for leap years
years_per_century = 100  # Number of years in a century
optimized_sigma = result.x[0] # The optimized value for sigma
optimized_eccentricity = result.x[1]*e

# Calculate the number of orbits Mercury completes in a century
orbits_per_century = (days_per_year / orbital_period_days) * years_per_century

# Calculate the precession per orbit in radians
precession_per_orbit_radians = 2 * np.pi * (optimized_sigma - 1)

# Convert precession per orbit to arcseconds
precession_per_orbit_arcseconds = np.degrees(precession_per_orbit_radians) * 3600

# Total precession in arcseconds per century
total_precession_arcseconds_per_century = precession_per_orbit_arcseconds * orbits_per_century
print(total_precession_arcseconds_per_century)


Optimized sigma: [1.00000008e+00 9.99999806e-01 2.85788598e+15]
42.22866927286121


In [28]:
print(f"h0= {h:.9e}")

h0= 2.712984359e+15
